In [ ]:
!pip install transformers datasets

In [ ]:
from datasets import load_dataset
import numpy as np

In [ ]:
raw_datasets = load_dataset("glue", "rte")

In [ ]:
raw_datasets

In [ ]:
raw_datasets["train"].features

In [ ]:
raw_datasets['train']['sentence1'][:10]

In [ ]:
checkpoint = "distilbert-base-cased"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenizer(
    raw_datasets['train']['sentence1'][0],
    raw_datasets['train']['sentence2'][0],
)

In [ ]:
result = _

# distilbert doesn't use token_type_ids
result.keys()

In [ ]:
tokenizer.decode(result['input_ids'])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels = 2
)

In [ ]:
training_args = TrainingArguments(
    output_dir='training_dir', 
    evaluation_strategy='epoch', 
    save_strategy='epoch', 
    num_train_epochs=5, 
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=64, 
    logging_steps=150, # otherwise no log will appear under training loss
)

In [ ]:
from datasets import load_metric 
metric = load_metric("glue", "rte")

In [ ]:
metric.compute(predictions=[1, 0, 1], references = [1, 0, 0])

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def compute_metrics(logits_and_labels): 
    logits, labels = logits_and_labels
    predictions = np.argmax(logits, axis=-1)
    acc = np.mean(predictions == labels) 
    f1 = f1_score(labels, predictions) 
    return {'accuracy': acc, 'f1': f1} 

In [ ]:
def tokenize_fn(batch): 
    return tokenizer(batch['sentence1'], batch['sentence2'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_fn, batched = True) 


In [ ]:
trainer = Trainer(
    model, 
    training_args, 
    train_dataset=tokenized_datasets['train'], 
    eval_dataset = tokenized_datasets['validation'], 
    tokenizer=tokenizer, 
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()